<b> Replication Notebooks for [The Impact of High School Financial Education: Evidence from a Large-Scale Evaluation in Brazil]( https://doi.org/10.1257/app.20150149) </b>

By [Charlie Zhang](mailto:charlie.zhang@georgetown.edu) and Hannah Roemer

In [1]:
import os
os.chdir("../")

import pandas as pd
import numpy as np
import statsmodels as sm
import statsmodels.formula.api as smf
import scipy

import warnings
warnings.filterwarnings("ignore")

In [2]:
panel_final = os.getcwd() + "/Data/school_intervention_panel_final.dta"

In [3]:
pf = pd.read_stata(panel_final)
pf.head(5)

,id_geral,cd_escola,nm_uf_bl,matriculas,docentes,abandonona1sriemdio,aprovaona1sriemdio,treatment,pair_all,treatment_workshop,...,dumm_rp_18p_fup,dumm_rp_19p_fup,dumm_rp_21p_fup,dumm_rp_23p_fup,dumm_formal_saving_fup,dumm_rp_33p_fup,dumm_rp_34p_fup,dumm_rp_36p_fup,dumm_rp_37p_fup,dumm_rp_41p_fup
0,1,17002648.0,TOCANTINS,273.0,34.0,6.8,76.7,yes,17017.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,17002648.0,TOCANTINS,273.0,34.0,6.8,76.7,yes,17017.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,17018390.0,TOCANTINS,641.0,29.0,4.2,90.7,no,17008.0,NaN,...,1.0,1.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,NaN
3,10,17018390.0,TOCANTINS,641.0,29.0,4.2,90.7,no,17008.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100,33002614.0,RIO DE JANEIRO,199.0,35.0,13.3,80.0,yes,33031.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,NaN


In [4]:
pf.columns

Index(['id_geral', 'cd_escola', 'nm_uf_bl', 'matriculas', 'docentes',
       'abandonona1sriemdio', 'aprovaona1sriemdio', 'treatment', 'pair_all',
       'treatment_workshop',
       ...
       'dumm_rp_18p_fup', 'dumm_rp_19p_fup', 'dumm_rp_21p_fup',
       'dumm_rp_23p_fup', 'dumm_formal_saving_fup', 'dumm_rp_33p_fup',
       'dumm_rp_34p_fup', 'dumm_rp_36p_fup', 'dumm_rp_37p_fup',
       'dumm_rp_41p_fup'],
      dtype='object', length=212)

In [5]:
variables = pd.read_stata(panel_final, iterator="True")
labels = variables.variable_labels()

In [6]:
variables.value_labels()

{'rp_41p_fup': {1: 'only expenses',
  2: 'only income',
  3: 'both',
  4: "don't know"},
 'rp_37p': {1: 'more than today',
  2: 'the sames as today',
  3: 'less than today',
  4: "don't know"},
 'rp_36p': {1: 'bank a', 2: 'bank b', 3: "don't know"},
 'rp_34p': {1: 'yes', 2: 'no'},
 'rp_33p': {1: 'yes', 2: 'no'},
 'rp_23p': {1: 'yes', 2: 'no', 3: "don't know"},
 'rp_21p': {1: 'yes', 2: 'no', 3: "don't know"},
 'rp_19p': {1: 'yes', 2: 'no', 3: "don't know"},
 'rp_18p': {1: 'yes', 2: 'no', 3: "don't know"},
 'rp_14p': {1: 'yes, always', 2: 'yes, somtimes', 3: 'no', 4: "don't know"},
 'rp_13p_fup': {1: 'nothing',
  2: 'up to 25%',
  3: 'between 26% and 50%',
  4: 'between 51% and 75%',
  5: 'between 76% and 100%'},
 'rp_09p': {1: 'formal employee',
  2: 'informal employee',
  3: 'businessperson',
  4: 'independent professional',
  5: 'self-employed',
  6: 'government employee',
  7: 'housewife',
  8: 'retired',
  9: 'pensioner',
  10: 'unemployed/no income',
  11: 'other'},
 'rp_08p': {1: 

## Table 1

In [7]:
# Line 41-43
pf_t1 = (pf[(pf["round"] == "yes")& (pf["treatment"].isna() == False)]
      .sort_values(by=["cd_escola", "id_geral"])
      .reset_index()
      .drop("index", axis=1))
pf_t1.head(5)

,id_geral,cd_escola,nm_uf_bl,matriculas,docentes,abandonona1sriemdio,aprovaona1sriemdio,treatment,pair_all,treatment_workshop,...,dumm_rp_18p_fup,dumm_rp_19p_fup,dumm_rp_21p_fup,dumm_rp_23p_fup,dumm_formal_saving_fup,dumm_rp_33p_fup,dumm_rp_34p_fup,dumm_rp_36p_fup,dumm_rp_37p_fup,dumm_rp_41p_fup
0,10166,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10439,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10448,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10568,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13397,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
test = [
    "female", "dumm_rp_08_bl", "dumm_rp_09_bl", "dumm_rp_24_bl",
    "dumm_rp_14_bl", "dumm_rp_23_bl", "vl_proficiencia_bl"
]
aluno = [
    "dumm_rp_49_bl", "dumm_rp_50_bl", "dumm_rp_65A_bl", "poupar_final2_bl",
    "dumm_rp_64A_bl", "dumm_negotiates_bl", "autonomia_final2_bl"
]
school = [
    "matriculas", "docentes", "abandonona1sriemdio", "aprovaona1sriemdio"
]
xvars = test + aluno

In [9]:
def ttest(stats, var, alpha=0.05):

    mean_1, mean_2 = stats[(var, "mean")]
    std_1, std_2 = stats[(var, "std")]
    n_1, n_2 = stats[(var, "count")]
    se_1, se_2 = std_1 / np.sqrt(n_1), std_2 / np.sqrt(n_2)
    sed = np.sqrt(se_1**2.0 + se_1**2.0)
    t_stat = (mean_1 - mean_2) / sed
    df = n_1 + n_2 - 2
    cv = scipy.stats.t.ppf(1.0 - alpha, df)
    p = (1.0 - scipy.stats.t.cdf(abs(t_stat), df)) * 2.0
    return p

In [10]:
t1p1 = pd.DataFrame()
for var in school:
    temp = ["treatment", "cd_escola"]
    temp.append(var)
    pf_temp = (pf_t1[temp].drop_duplicates()[["treatment", var]])
    stats = pf_temp.groupby("treatment").agg({var: ["mean", "std", "count"]})
    stats[(var, "t-test")] = ttest(stats, var)
    t1p1 = pd.concat([t1p1, stats], axis=1).round(4)

display(t1p1)

matriculas                         docentes                         \
                mean       std count  t-test     mean      std count  t-test   
treatment                                                                      
no          642.8363  461.6997   452  0.2156  37.4949  24.0912   392  0.6504   
yes         680.8932  514.6180   440  0.2156  38.2751  25.7419   378  0.6504   

          abandonona1sriemdio                        aprovaona1sriemdio  \
                         mean      std count  t-test               mean   
treatment                                                                 
no                    11.0351  11.2233   447  0.3488            68.0510   
yes                   11.7389  11.8174   435  0.3488            67.7446   

                                  
               std count  t-test  
treatment                         
no         16.0285   447  0.7751  
yes        15.9359   435  0.7751

In [11]:
t1p2 = pd.DataFrame()
for var in xvars:
    temp = ["treatment", "id_geral"]
    temp.append(var)
    pf_temp = (pf_t1[temp].drop_duplicates()[["treatment", var]])
    stats = pf_temp.groupby("treatment").agg({var: ["mean", "std", "count"]})
    stats[(var, "t-test")] = ttest(stats, var)
    t1p2 = pd.concat([t1p2, stats], axis=1).round(3)

display(t1p2.T)

treatment                             no           yes
female              mean        0.549000      0.563000
                    std         0.498000      0.496000
                    count   11954.000000  11529.000000
                    t-test      0.034000      0.034000
dumm_rp_08_bl       mean        0.446000      0.435000
                    std         0.497000      0.496000
                    count   11774.000000  11350.000000
                    t-test      0.111000      0.111000
dumm_rp_09_bl       mean        0.414000      0.405000
                    std         0.493000      0.491000
                    count   11720.000000  11283.000000
                    t-test      0.133000      0.133000
dumm_rp_24_bl       mean        0.297000      0.315000
                    std         0.457000      0.465000
                    count   11741.000000  11335.000000
                    t-test      0.002000      0.002000
dumm_rp_14_bl       mean        0.316000      0.333000
                    std         0.465000      0.471000
                    count   11878.000000  11435.000000
                    t-test      0.004000      0.004000
dumm_rp_23_bl       mean        0.515000      0.529000
                    std         0.500000      0.499000
                    count   11783.000000  11363.000000
                    t-test      0.037000      0.037000
vl_proficiencia_bl  mean       49.868000     50.153000
                    std        15.163000     14.830000
                    count   12173.000000  11751.000000
                    t-test      0.142000      0.142000
dumm_rp_49_bl       mean        0.352000      0.351000
                    std         0.478000      0.477000
                    count   12769.000000  12229.000000
                    t-test      0.871000      0.871000
dumm_rp_50_bl       mean        0.661000      0.672000
                    std         0.473000      0.469000
                    count   12778.000000  12234.000000
                    t-test      0.073000      0.073000
dumm_rp_65A_bl      mean        0.153000      0.161000
                    std         0.360000      0.367000
                    count   12533.000000  12003.000000
                    t-test      0.092000      0.092000
poupar_final2_bl    mean       48.360001     48.175999
                    std        18.911000     18.557000
                    count   11985.000000  11479.000000
                    t-test      0.453000      0.453000
dumm_rp_64A_bl      mean        0.103000      0.105000
                    std         0.304000      0.307000
                    count   12657.000000  12109.000000
                    t-test      0.566000      0.566000
dumm_negotiates_bl  mean        0.753000      0.758000
                    std         0.431000      0.429000
                    count   12525.000000  12030.000000
                    t-test      0.421000      0.421000
autonomia_final2_bl mean       49.077000     48.981998
                    std        19.806000     19.582000
                    count   12005.000000  11445.000000
                    t-test      0.711000      0.711000

In [12]:
t1p2_cols = t1p2.columns.to_list()
t1p2_new_cols = []
for col in t1p2_cols:
    a, b = col
    a = labels[a]
    t1p2_new_cols.append((a,b))
t1p2.columns = pd.MultiIndex.from_tuples(t1p2_new_cols)

In [13]:
t1p2.T.round(2)

treatment                                                        no       yes
Student is female                                  mean        0.55      0.56
                                                   std         0.50      0.50
                                                   count   11954.00  11529.00
                                                   t-test      0.03      0.03
Education of student's mother: At least some se... mean        0.45      0.44
                                                   std         0.50      0.50
                                                   count   11774.00  11350.00
                                                   t-test      0.11      0.11
Education of student's father: At least some se... mean        0.41      0.41
                                                   std         0.49      0.49
                                                   count   11720.00  11283.00
                                                   t-test      0.13      0.13
Student has failed at least one school year        mean        0.30      0.31
                                                   std         0.46      0.46
                                                   count   11741.00  11335.00
                                                   t-test      0.00      0.00
Student's family receives Bolsa Familia cash tr... mean        0.32      0.33
                                                   std         0.46      0.47
                                                   count   11878.00  11435.00
                                                   t-test      0.00      0.00
Student has computer with internet at home         mean        0.51      0.53
                                                   std         0.50      0.50
                                                   count   11783.00  11363.00
                                                   t-test      0.04      0.04
Student financial literacy score                   mean       49.87     50.15
                                                   std        15.16     14.83
                                                   count   12173.00  11751.00
                                                   t-test      0.14      0.14
Student is not working at the moment               mean        0.35      0.35
                                                   std         0.48      0.48
                                                   count   12769.00  12229.00
                                                   t-test      0.87      0.87
Q50 Receives income                                mean        0.66      0.67
                                                   std         0.47      0.47
                                                   count   12778.00  12234.00
                                                   t-test      0.07      0.07
Q65 Saves money for future purchases               mean        0.15      0.16
                                                   std         0.36      0.37
                                                   count   12533.00  12003.00
                                                   t-test      0.09      0.09
Student intention to save index                    mean       48.36     48.18
                                                   std        18.91     18.56
                                                   count   11985.00  11479.00
                                                   t-test      0.45      0.45
Q64 I make a list of all monthly expenses          mean        0.10      0.10
                                                   std         0.30      0.31
                                                   count   12657.00  12109.00
                                                   t-test      0.57      0.57
Q93 Q95: Negotiates prices or payment methods      mean        0.75      0.76
                                                   std         0.43      0.43
                      

## Table 2

In [14]:
# Line 199-201
pf_t2 = (pf[(pf["round"] == "no") & (pf["treatment"].isna() == False) &
            (pf["fu1_test"] == 1) | (pf["fu1_aluno"] == 1)]
         .sort_values(by=["cd_escola", "id_geral"])
         .reset_index()
         .drop("index",axis=1))
pf_t2.head(5)

,id_geral,cd_escola,nm_uf_bl,matriculas,docentes,abandonona1sriemdio,aprovaona1sriemdio,treatment,pair_all,treatment_workshop,...,dumm_rp_18p_fup,dumm_rp_19p_fup,dumm_rp_21p_fup,dumm_rp_23p_fup,dumm_formal_saving_fup,dumm_rp_33p_fup,dumm_rp_34p_fup,dumm_rp_36p_fup,dumm_rp_37p_fup,dumm_rp_41p_fup
0,10166,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,1.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0,0.0,NaN
1,10439,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
2,10448,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
3,10568,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN
4,13397,17000386.0,TOCANTINS,414.0,42.0,9.6,65.2,yes,17006.0,NaN,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,NaN


In [15]:
pf_t2["female_coded"] = pf_t2["female_coded"].map({"yes": 1, "no": 0,})
pf_t2["female_coded"].value_counts()

1.0    11029
0.0     8685
Name: female_coded, dtype: int64

In [16]:
responsavel = ["p_employee_fup", "p_selfempl_fup", "p_other_fup"]
for var in responsavel:
    pf_temp = pf_t2[["treatment", "cd_escola", var]].dropna().reset_index().drop("index", axis=1)
    var_res = smf.ols(str(var) + "~treatment", data=pf_temp).fit(
        cov_type='cluster', cov_kwds={'groups': pf_temp["cd_escola"]})
    display(var_res.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         p_employee_fup   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                   0.04533
Date:                Mon, 17 Jul 2023   Prob (F-statistic):              0.831
Time:                        16:42:08   Log-Likelihood:                -9817.9
No. Observations:               13540   AIC:                         1.964e+04
Df Residuals:                   13538   BIC:                         1.965e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.4802      0.008     59.314      0.000       0.464       0.496
treatment[T.yes]     0.0024      0.011      0.213      0.831      -0.020       0.025
==============================================================================
Omnibus:                    46608.087   Durbin-Watson:                   1.925
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2256.631
Skew:                           0.074   Prob(JB):                         0.00
Kurtosis:                       1.005   Cond. No.                         2.71
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         p_selfempl_fup   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                   0.03395
Date:                Mon, 17 Jul 2023   Prob (F-statistic):              0.854
Time:                        16:42:08   Log-Likelihood:                -6226.6
No. Observations:               13540   AIC:                         1.246e+04
Df Residuals:                   13538   BIC:                         1.247e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.1796      0.005     33.476      0.000       0.169       0.190
treatment[T.yes]    -0.0014      0.008     -0.184      0.854      -0.016       0.014
==============================================================================
Omnibus:                     3436.679   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6705.819
Skew:                           1.676   Prob(JB):                         0.00
Kurtosis:                       3.808   Cond. No.                         2.71
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            p_other_fup   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                  0.008471
Date:                Mon, 17 Jul 2023   Prob (F-statistic):              0.927
Time:                        16:42:08   Log-Likelihood:                -9091.8
No. Observations:               13540   AIC:                         1.819e+04
Df Residuals:                   13538   BIC:                         1.820e+04
Df Model:                           1                                         
Covariance Type:              cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            0.3401      0.008     43.014      0.000       0.325       0.356
treatment[T.yes]    -0.0010      0.011     -0.092      0.927      -0.023       0.021
==============================================================================
Omnibus:                    75465.707   Durbin-Watson:                   1.919
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2375.496
Skew:                           0.677   Prob(JB):                         0.00
Kurtosis:                       1.459   Cond. No.                         2.71
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

,treatment,cd_escola,female_coded
0,1,17002648.0,0.0
1,1,17002648.0,0.0
2,0,17018390.0,0.0
3,0,17018390.0,0.0
4,1,33002614.0,1.0
...,...,...,...
70503,0,53007514.0,1.0
70504,1,33075212.0,1.0
70505,1,33075212.0,1.0
70506,0,35907637.0,1.0


In [23]:
pf_temp = pf[["treatment", "cd_escola", "female_coded"]]
pf_temp = pf_temp[pf_temp.female_coded != "missing"].reset_index().drop("index", axis=1)
pf_temp["female_coded"] = pf_temp["female_coded"].map({"yes": 1, "no": 0})
pf_temp["treatment"] = pf_temp["treatment"].map({"yes": 1, "no": 0})
pf_temp = pf_temp.dropna()
fc_res = smf.ols("female_coded~treatment", data=pf_temp).fit(cov_type='cluster', 
                                                             cov_kwds={'groups': pf_temp["cd_escola"]})
fc_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           female_coded   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     5.573
Date:                Mon, 17 Jul 2023   Prob (F-statistic):             0.0185
Time:                        16:44:04   Log-Likelihood:                -50600.
No. Observations:               70480   AIC:                         1.012e+05
Df Residuals:                   70478   BIC:                         1.012e+05
Df Model:                           1                                         
Covariance Type:              cluster                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.5542      0.005    122.972      0.000       0.545       0.563
treatment[T.1]     0.0155      0.007      2.361      0.018       0.003       0.028
==============================================================================
Omnibus:                   250188.183   Durbin-Watson:                   0.566
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11746.753
Skew:                          -0.250   Prob(JB):                         0.00
Kurtosis:                       1.063   Cond. No.                         2.61
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

## Table 3

In [24]:
incl_vars = ["vl_proficiencia_fup", "treatment", "cd_escola"]
for rd in ["no", "yes"]:
    data = (pf[pf["round"] == rd][incl_vars]
            .dropna().reset_index().drop("index", axis=1))
    school_id = data["cd_escola"]
    t3_nc = smf.ols("vl_proficiencia_fup~treatment", data=data)
    t3_nc_res = t3_nc.fit(cov_type='cluster', cov_kwds={'groups': school_id})
    display(t3_nc_res.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     vl_proficiencia_fup   R-squared:                       0.021
Model:                             OLS   Adj. R-squared:                  0.021
Method:                  Least Squares   F-statistic:                     55.23
Date:                 Mon, 17 Jul 2023   Prob (F-statistic):           2.61e-13
Time:                         16:44:20   Log-Likelihood:                -74895.
No. Observations:                18276   AIC:                         1.498e+05
Df Residuals:                    18274   BIC:                         1.498e+05
Df Model:                            1                                         
Covariance Type:               cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           56.0501      0.425    131.941      0.000      55.217      56.883
treatment[T.yes]     4.2661      0.574      7.432      0.000       3.141       5.391
==============================================================================
Omnibus:                      729.291   Durbin-Watson:                   1.922
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              317.535
Skew:                           0.039   Prob(JB):                     1.12e-69
Kurtosis:                       2.359   Cond. No.                         2.65
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     vl_proficiencia_fup   R-squared:                       0.012
Model:                             OLS   Adj. R-squared:                  0.012
Method:                  Least Squares   F-statistic:                     32.93
Date:                 Mon, 17 Jul 2023   Prob (F-statistic):           1.33e-08
Time:                         16:44:20   Log-Likelihood:                -77954.
No. Observations:                18953   AIC:                         1.559e+05
Df Residuals:                    18951   BIC:                         1.559e+05
Df Model:                            1                                         
Covariance Type:               cluster                                         
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           59.0446      0.409    144.341      0.000      58.243      59.846
treatment[T.yes]     3.2938      0.574      5.739      0.000       2.169       4.419
==============================================================================
Omnibus:                      748.943   Durbin-Watson:                   1.822
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              342.039
Skew:                           0.089   Prob(JB):                     5.34e-75
Kurtosis:                       2.367   Cond. No.                         2.63
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [25]:
pf = pd.read_stata(panel_final)
for rd in ["no", "yes"]:
    incl_vars_ctrl = [
        "vl_proficiencia_fup", "treatment", "cd_escola", "pair_all"
    ]
    data = (pf[incl_vars_ctrl][(
        pf["round"] == rd)].dropna().reset_index().drop("index", axis=1))
    data["treatment"] = data["treatment"].map({
        "yes": 1,
        "no": 0
    }).astype("float")
   
    flag_temp = pd.DataFrame(
        data.groupby("pair_all")["treatment"].mean().reset_index().rename(
            {"treatment": "flag"}, axis=1))
    data = data.merge(flag_temp, how="left", on="pair_all")

    pairall = []
    for par, flg in zip(data["pair_all"], data["flag"]):
        if (flg > 0) & (flg < 1):
            pairall.append(par)
        else:
            pairall.append(0)
    data["pair_all"] = pairall

    t3_c = smf.ols("vl_proficiencia_fup~treatment+C(pair_all)", data=data)
    t3_c_res = t3_c.fit(cov_type='cluster',
                        cov_kwds={'groups': data["cd_escola"]})
    display(t3_c_res.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     vl_proficiencia_fup   R-squared:                       0.182
Model:                             OLS   Adj. R-squared:                  0.164
Method:                  Least Squares   F-statistic:                 1.467e+04
Date:                 Mon, 17 Jul 2023   Prob (F-statistic):               0.00
Time:                         16:44:26   Log-Likelihood:                -73257.
No. Observations:                18276   AIC:                         1.473e+05
Df Residuals:                    17882   BIC:                         1.504e+05
Df Model:                          393                                         
Covariance Type:               cluster                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  56.2995      1.090     51.665      0.000      54.164      58.435
C(pair_all)[T.17001.0]     -0.6258      1.187     -0.527      0.598      -2.953       1.701
C(pair_all)[T.17002.0]     -1.3422      2.519     -0.533      0.594      -6.279       3.594
C(pair_all)[T.17003.0]     -1.5951      4.139     -0.385      0.700      -9.708       6.518
C(pair_all)[T.17004.0]     -1.6417      3.071     -0.535      0.593      -7.661       4.377
C(pair_all)[T.17005.0]     -4.7960      1.980     -2.422      0.015      -8.677      -0.915
C(pair_all)[T.17006.0]     -3.6116      4.103     -0.880      0.379     -11.653       4.429
C(pair_all)[T.17007.0]      1.4525      1.769      0.821      0.412      -2.014       4.919
C(pair_all)[T.17008.0]     -0.3619      1.097     -0.330      0.742      -2.513       1.789
C(pair_all)[T.17009.0]     -5.4989      2.929     -1.877      0.060     -11.240       0.242
C(pair_all)[T.17010.0]     -3.1781      5.972     -0.532      0.595     -14.883       8.526
C(pair_all)[T.17011.0]      2.6173      2.585      1.012      0.311      -2.449       7.684
C(pair_all)[T.17012.0]      7.5221      2.201      3.418      0.001       3.209      11.835
C(pair_all)[T.17013.0]      1.9125      3.149      0.607      0.544      -4.259       8.084
C(pair_all)[T.17014.0]     -3.1602      1.681     -1.880      0.060      -6.455       0.134
C(pair_all)[T.17015.0]      3.5217      1.994      1.767      0.077      -0.386       7.429
C(pair_all)[T.17016.0]     -6.5710      1.097     -5.992      0.000      -8.720      -4.422
C(pair_all)[T.17017.0]      2.9414      1.424      2.065      0.039       0.150       5.733
C(pair_all)[T.23001.0]      7.3701      4.949      1.489      0.136      -2.331      17.071
C(pair_all)[T.23003.0]     -5.4559      2.118     -2.575      0.010      -9.608      -1.304
C(pair_all)[T.23004.0]     -0.1368      2.381     -0.057      0.954      -4.804       4.531
C(pair_all)[T.23005.0]      8.3867      1.099      7.633      0.000       6.233      10.540
C(pair_all)[T.23006.0]     -7.4459      2.347     -3.173      0.002     -12.046      -2.846
C(pair_all)[T.23007.0]      8.8019      1.694      5.195      0.000       5.481      12.123
C(pair_all)[T.23008.0]     -4.5094      1.120     -4.028      0.000      -6.704      -2.315
C(pair_all)[T.23009.0]     -4.3897      2.108     -2.083      0.037      -8.521      -0.258
C(pair_all)[T.23010.0]      2.3740      1.446      1.642      0.101      -0.459       5.207
C(pair_all)[T.23011.0]     -5.0478      3.570     -1.414      0.157     -12.046       1.950
C(pair_all)[T.23012.0]      2.2872      2.629      0.870      0.384      -2.865       7.440
C(pair_all)[T.23013.0]      0.2504      6.550      0.038      0.970     -12.586      13.087
C(pair_all)[T.23014.0]     -9.0932  

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     vl_proficiencia_fup   R-squared:                       0.171
Model:                             OLS   Adj. R-squared:                  0.154
Method:                  Least Squares   F-statistic:                     277.3
Date:                 Mon, 17 Jul 2023   Prob (F-statistic):               0.00
Time:                         16:44:27   Log-Likelihood:                -76290.
No. Observations:                18953   AIC:                         1.534e+05
Df Residuals:                    18561   BIC:                         1.564e+05
Df Model:                          391                                         
Covariance Type:               cluster                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  57.7120      1.092     52.863      0.000      55.572      59.852
C(pair_all)[T.17001.0]      4.2310      1.069      3.957      0.000       2.135       6.327
C(pair_all)[T.17002.0]      2.1128      4.990      0.423      0.672      -7.667      11.893
C(pair_all)[T.17003.0]     -2.0471      3.675     -0.557      0.578      -9.250       5.156
C(pair_all)[T.17004.0]     -0.0948      3.492     -0.027      0.978      -6.939       6.749
C(pair_all)[T.17005.0]      0.4167      5.842      0.071      0.943     -11.033      11.867
C(pair_all)[T.17006.0]      2.3491      2.457      0.956      0.339      -2.466       7.164
C(pair_all)[T.17007.0]      2.4467      2.034      1.203      0.229      -1.539       6.433
C(pair_all)[T.17008.0]     -5.0975      6.284     -0.811      0.417     -17.413       7.218
C(pair_all)[T.17009.0]     -2.0084      3.608     -0.557      0.578      -9.081       5.064
C(pair_all)[T.17010.0]     -3.2327      2.140     -1.511      0.131      -7.426       0.961
C(pair_all)[T.17011.0]      1.4303      3.481      0.411      0.681      -5.392       8.253
C(pair_all)[T.17012.0]      2.6364      1.593      1.655      0.098      -0.486       5.759
C(pair_all)[T.17013.0]      5.0643      2.386      2.123      0.034       0.389       9.740
C(pair_all)[T.17014.0]      4.8734      1.266      3.849      0.000       2.392       7.355
C(pair_all)[T.17016.0]    -14.6607      1.409    -10.407      0.000     -17.422     -11.900
C(pair_all)[T.17017.0]      5.8222      1.120      5.196      0.000       3.626       8.018
C(pair_all)[T.23001.0]      4.9001      1.394      3.515      0.000       2.168       7.632
C(pair_all)[T.23003.0]     -0.3259      1.216     -0.268      0.789      -2.710       2.058
C(pair_all)[T.23004.0]     -0.4303      4.568     -0.094      0.925      -9.383       8.523
C(pair_all)[T.23005.0]      9.5953      5.043      1.903      0.057      -0.289      19.480
C(pair_all)[T.23006.0]     -6.8610      3.056     -2.245      0.025     -12.850      -0.872
C(pair_all)[T.23007.0]     12.0887      4.329      2.793      0.005       3.605      20.573
C(pair_all)[T.23008.0]      1.3524      1.352      1.000      0.317      -1.298       4.003
C(pair_all)[T.23009.0]     -7.2051      5.126     -1.405      0.160     -17.253       2.843
C(pair_all)[T.23010.0]      5.0603      1.710      2.959      0.003       1.709       8.412
C(pair_all)[T.23011.0]     -6.3317      4.564     -1.387      0.165     -15.277       2.614
C(pair_all)[T.23012.0]      2.1155      2.313      0.914      0.360      -2.419       6.650
C(pair_all)[T.23013.0]      1.5253      4.230      0.361      0.718      -6.766       9.817
C(pair_all)[T.23014.0]     -4.5290      3.035     -1.492      0.136     -10.477       1.419
C(pair_all)[T.23015.0]      5.3503  

In [27]:
pf = pd.read_stata(panel_final)
incl_vars_ctrl = [
    "vl_proficiencia_fup",
    "vl_proficiencia_bl",
    "treatment",
    "cd_escola",
    "pair_all",
    "female_coded"  ##, "p_employee_fup", "p_selfempl_fup",
]
data = (
    pf[incl_vars_ctrl][(pf["round"] == "no")].reset_index().drop(
                           "index", axis=1))

data["female_coded"] = data["female_coded"].map({
    "yes": 1,
    "no": 0,
    "missing": 9
}).astype("float")

data["treatment"] = data["treatment"].map({"yes": 1, "no": 0}).astype("float")

flag_temp = pd.DataFrame(
    data.groupby("pair_all")["treatment"].mean().reset_index().rename(
        {"treatment": "flag"}, axis=1))
data = data.merge(flag_temp, how="left", on="pair_all")

data["miss_vl_proficiencia_bl"] = np.where(
    data["vl_proficiencia_bl"].isna() == True, 1, 0)
data["vl_proficiencia_bl"] = data["miss_vl_proficiencia_bl"].fillna(0)

pairall = []
for par, flg in zip(data["pair_all"], data["flag"]):
    if (flg > 0) & (flg < 1):
        pairall.append(par)
    elif flg == 0:
        pairall.append(0)
    elif flg == 1:
        pairall.append(0)
    else:
        pairall.append(0)
data["pair_all"] = pairall

data = data.dropna().reset_index().drop("index", axis=1)
t3_c = smf.ols(
    "vl_proficiencia_fup~treatment+C(pair_all)+female_coded+vl_proficiencia_bl+miss_vl_proficiencia_bl",
    data=data)
t3_c_res = t3_c.fit(cov_type='cluster', cov_kwds={'groups': data["cd_escola"]})
display(t3_c_res.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     vl_proficiencia_fup   R-squared:                       0.204
Model:                             OLS   Adj. R-squared:                  0.185
Method:                  Least Squares   F-statistic:                    0.1307
Date:                 Mon, 17 Jul 2023   Prob (F-statistic):              0.877
Time:                         16:47:41   Log-Likelihood:                -73005.
No. Observations:                18276   AIC:                         1.469e+05
Df Residuals:                    17843   BIC:                         1.503e+05
Df Model:                          432                                         
Covariance Type:               cluster                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  53.3332      1.649     32.347      0.000      50.102      56.565
C(pair_all)[T.17001.0]      1.5004      1.711      0.877      0.380      -1.852       4.853
C(pair_all)[T.17002.0]      0.8434      2.792      0.302      0.763      -4.629       6.315
C(pair_all)[T.17003.0]      0.2494      4.158      0.060      0.952      -7.901       8.399
C(pair_all)[T.17004.0]      0.4954      3.167      0.156      0.876      -5.712       6.703
C(pair_all)[T.17005.0]     -2.5060      2.314     -1.083      0.279      -7.041       2.029
C(pair_all)[T.17006.0]     -1.8912      4.189     -0.451      0.652     -10.102       6.319
C(pair_all)[T.17007.0]      3.6646      2.093      1.751      0.080      -0.438       7.767
C(pair_all)[T.17008.0]      1.8096      1.645      1.100      0.271      -1.415       5.035
C(pair_all)[T.17009.0]     -3.0834      3.148     -0.980      0.327      -9.253       3.086
C(pair_all)[T.17010.0]     -0.8958      5.898     -0.152      0.879     -12.456      10.665
C(pair_all)[T.17011.0]      4.8193      3.116      1.547      0.122      -1.288      10.927
C(pair_all)[T.17012.0]      9.0511      2.286      3.959      0.000       4.571      13.531
C(pair_all)[T.17013.0]      3.6413      3.321      1.096      0.273      -2.868      10.151
C(pair_all)[T.17014.0]     -1.5036      2.070     -0.726      0.468      -5.561       2.553
C(pair_all)[T.17015.0]      5.0614      2.769      1.828      0.068      -0.365      10.488
C(pair_all)[T.17016.0]     -4.0185      1.641     -2.449      0.014      -7.234      -0.803
C(pair_all)[T.17017.0]      4.9933      1.964      2.542      0.011       1.143       8.844
C(pair_all)[T.23001.0]      9.3527      5.006      1.868      0.062      -0.459      19.165
C(pair_all)[T.23003.0]     -3.4496      2.535     -1.361      0.174      -8.418       1.519
C(pair_all)[T.23004.0]      2.1126      2.542      0.831      0.406      -2.869       7.095
C(pair_all)[T.23005.0]     10.1118      1.637      6.177      0.000       6.903      13.320
C(pair_all)[T.23006.0]     -5.1188      2.671     -1.916      0.055     -10.354       0.117
C(pair_all)[T.23007.0]     10.5583      2.189      4.823      0.000       6.268      14.849
C(pair_all)[T.23008.0]     -2.6173      1.651     -1.585      0.113      -5.854       0.619
C(pair_all)[T.23009.0]     -1.9205      2.346     -0.819      0.413      -6.518       2.677
C(pair_all)[T.23010.0]      3.9457      1.769      2.230      0.026       0.478       7.414
C(pair_all)[T.23011.0]     -2.8467      3.549     -0.802      0.423      -9.803       4.110
C(pair_all)[T.23012.0]      4.4400      3.275      1.356      0.175      -1.979      10.859
C(pair_all)[T.23013.0]      2.2788      6.621      0.344      0.731     -10.699      15.257
C(pair_all)[T.23014.0]     -6.8083  